# Constrained Solutions with Linear Constraints

In [1]:
from os.path import exists
file_exists = exists('basic_material.py')

if (not file_exists):
  !wget -O basic_material.py https://www.dropbox.com/s/3aoexl8vxyitmds/basic_material.py?dl=0
%run basic_material

try:
    import google.colab
    IN_COLAB = True
    # don't need serial here
except:
    IN_COLAB = False
    try: 
        import serial
    except:
        !pip3 install pyserial
        import serial

import time, csv, os, requests
 
from os import path
if os.path.isdir("./data/"):
    pass
else:
    !mkdir ./data
    
if os.path.isdir("./figs/"):
    pass
else:
    !mkdir ./figs

Running Python: 3.8.12


## Example 1 

Simple objective that can be made more interesting with $a \neq 0$.

There are simple upper and lower bounds on the feasible region. Changing $lb$ and $ub$ will modify which, if any, of these 2 constraints is active. 

In [2]:
#Create initial point.
x0 = np.array([2,2])

In [3]:
# constraint matrix
A = np.array([[-1,1]])
xoffset = - 0.25
yoffset = 0.25

Range = 3
N_samples = 25
x = np.arange(-Range, Range, 1/N_samples)
y = np.arange(-Range, Range, 1/N_samples)
X, Y = np.meshgrid(x, y)

# Objective ftn - shape controllable by choice of a
def f(X,a,b = 1):
    x,y = X
    x = x + xoffset 
    y = y + yoffset
    r = x**2 + a*x*y + b*y**2
    return (r)

In [4]:
# interactively plot result with constraint ftn
print("Variable a changes orientation of the Objective ftn")
print("lb and ub change the Constraint ftn")
def func(a = 0,bb = 0,lb = -1,ub = 1):
    b = 4**bb
    # make sure the upperbd > lowerbd
    lb = min(lb, ub - 1e-3) # make sure that lb <= ub
    ub = max(ub, lb + 1e-3) # make sure that lb <= ub
    r = f(np.array([X,Y]),a,b) 

    fig = plt.figure(1,figsize=(8,8))

    ax = fig.add_subplot()
    CS = ax.contour(X, Y, r,[0,.1,.5,1,2,3,4,8,16])
    Yub = (ub - A[0,0]*x)/A[0,1] # equation for the constraint line 
    Ylb = (lb - A[0,0]*x)/A[0,1] # equation for the constraint line 
    ax.plot(x,Yub,'r',lw = 2)
    ax.plot(x,Ylb,'g',lw = 2)

    ax.clabel(CS, inline=1, fontsize=10)
    ax.set_title('Contour')
    ax.set_ylim([-Range, Range])
    ax.set_xlim([-Range, Range])
    ax.set_aspect('equal')

    print('Problem data: a = '+str(a),', lb = '+str(lb),', ub = '+str(ub))
    lin_cons = LinearConstraint(A,lb,ub)
    sol_ineq = minimize(f,x0,constraints=lin_cons,args=(a,b), method='trust-constr', options={'verbose':0})
    #print(sol_ineq.v[0][0])
    ax.plot(sol_ineq.x[0], sol_ineq.x[1], 'bo')
    if (abs(sol_ineq.v[0][0]) < 1e-4): # Lagrange variable is near zero
        txt="Figure 1: Unconstrained min at ["+str(xoffset)+" , "+str(yoffset)+"]. \n Moving bounds above or below that leads to a \n constrained solution"
    else:      
        txt="Figure 1: Unconstrained min at ["+str(xoffset)+" , "+str(yoffset)+"]. \n Moving bounds above or below that leads to the \n constrained solution: "+str(sol_ineq.x)
    plt.figtext(0.5, -0.05, txt, wrap=True, horizontalalignment='center', fontsize=16)
    plt.grid(True)
    plt.show()
    return 0

y=interactive(func,a=(-1.95,1.95,0.05),bb=(-2,2,0.5),lb=(-2,3,0.1),ub=(-2,3,0.1))
y

Variable a changes orientation of the Objective ftn
lb and ub change the Constraint ftn


interactive(children=(FloatSlider(value=0.0, description='a', max=1.95, min=-1.95, step=0.05), FloatSlider(val…